<a href="https://colab.research.google.com/github/jeffinthomas/Dash/blob/master/Hypercube_test_jeffin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [16]:
# Load CSV and JSON datasets

df_bmrs = pd.read_csv('/content/drive/MyDrive/demo_data/bmrs_wind_forecast_pair.csv')
#df_bmrs.head()


# List of the columns you want to select
columns_to_select = [
    'recordType', 'startTimeOfHalfHrPeriod', 'settlementPeriod',
    'initialForecastPublishingPeriodCommencingTime', 'initialForecastSpnGeneration',
    'latestForecastPublishingPeriodCommencingTime', 'latestForecastSpnGeneration',
    'outTurnPublishingPeriodCommencingTime', 'fuelTypeGeneration', 'activeFlag', 'EFA'
]

# Select the specified columns from df_bmrs
df_bmrs = df_bmrs[columns_to_select]

print(len(df_bmrs))
# Display the resulting DataFrame
df_bmrs.head()



1415


,recordType,startTimeOfHalfHrPeriod,settlementPeriod,initialForecastPublishingPeriodCommencingTime,initialForecastSpnGeneration,latestForecastPublishingPeriodCommencingTime,latestForecastSpnGeneration,outTurnPublishingPeriodCommencingTime,fuelTypeGeneration,activeFlag,EFA
0,WIND,01/11/2021,1,30/10/2021 03:30,9643,31/10/2021 22:30,9944.0,01/11/2021 00:30,10972.0,Y,1.0
1,WIND,01/11/2021,2,NaN,NaN,NaN,NaN,01/11/2021 01:00,11053.0,Y,1.0
2,WIND,01/11/2021,3,30/10/2021 03:30,9773,31/10/2021 22:30,9920.0,01/11/2021 01:30,11030.0,Y,1.0
3,WIND,01/11/2021,4,NaN,NaN,NaN,NaN,01/11/2021 02:00,10787.0,Y,1.0
4,WIND,01/11/2021,5,30/10/2021 03:30,9919,31/10/2021 22:30,10029.0,01/11/2021 02:30,10806.0,Y,1.0


In [ ]:
##Athena query
#select count(distinct(settlementperiod))FROM "hypercube-demo-db"."source" limit 10; --49
#select distinct(settlementperiod) FROM "hypercube-demo-db"."source" limit 100;

#SELECT * FROM "hypercube-demo-db"."source" where settlementperiod = -1 limit 10; -- 3 counts


In [20]:
#settlementperiod will be 1 to 48
#droping '-1' from df
#print(df_bmrs.head())

df_bmrs = df_bmrs[df_bmrs['settlementPeriod'] > -1]

print(len(df_bmrs))

1412


In [22]:
import pandas as pd
from datetime import timedelta


# Convert 'startTimeOfHalfHrPeriod' to datetime object (this is the day in question)
df_bmrs['startTimeOfHalfHrPeriod'] = pd.to_datetime(df_bmrs['startTimeOfHalfHrPeriod'], format='%d/%m/%Y')

# Function to create the desired datetime string
def combine_date_and_settlement(df):
    # Adjust for the start of the first settlement period (which is 23:00 on the previous day)
    df['combinedDatetime'] = df['startTimeOfHalfHrPeriod'] - timedelta(hours=1) + pd.to_timedelta((df['settlementPeriod'] - 1) * 30, unit='min')
    df['combinedDatetime'] = df['combinedDatetime'].dt.strftime('%Y-%m-%dT%H:%M:%S')
    return df

# Apply the function to create the new column
df_bmrs = combine_date_and_settlement(df_bmrs)

# Print the final DataFrame with the new combined column
print(df_bmrs)

# Display the resulting DataFrame
df_bmrs.head()

     recordType startTimeOfHalfHrPeriod  settlementPeriod  \
0          WIND              2021-11-01                 1   
1          WIND              2021-11-01                 2   
2          WIND              2021-11-01                 3   
3          WIND              2021-11-01                 4   
4          WIND              2021-11-01                 5   
...         ...                     ...               ...   
1410       WIND              2021-11-27                39   
1411       WIND              2021-11-27                41   
1412       WIND              2021-11-27                43   
1413       WIND              2021-11-27                45   
1414       WIND              2021-11-27                47   

     initialForecastPublishingPeriodCommencingTime  \
0                                 30/10/2021 03:30   
1                                              NaN   
2                                 30/10/2021 03:30   
3                                              NaN 

,recordType,startTimeOfHalfHrPeriod,settlementPeriod,initialForecastPublishingPeriodCommencingTime,initialForecastSpnGeneration,latestForecastPublishingPeriodCommencingTime,latestForecastSpnGeneration,outTurnPublishingPeriodCommencingTime,fuelTypeGeneration,activeFlag,EFA,combinedDatetime
0,WIND,2021-11-01,1,30/10/2021 03:30,9643,31/10/2021 22:30,9944.0,01/11/2021 00:30,10972.0,Y,1.0,2021-10-31T23:00:00
1,WIND,2021-11-01,2,NaN,NaN,NaN,NaN,01/11/2021 01:00,11053.0,Y,1.0,2021-10-31T23:30:00
2,WIND,2021-11-01,3,30/10/2021 03:30,9773,31/10/2021 22:30,9920.0,01/11/2021 01:30,11030.0,Y,1.0,2021-11-01T00:00:00
3,WIND,2021-11-01,4,NaN,NaN,NaN,NaN,01/11/2021 02:00,10787.0,Y,1.0,2021-11-01T00:30:00
4,WIND,2021-11-01,5,30/10/2021 03:30,9919,31/10/2021 22:30,10029.0,01/11/2021 02:30,10806.0,Y,1.0,2021-11-01T01:00:00


In [23]:
# Load the JSON file into a DataFrame
df_orders = pd.read_json('/content/drive/MyDrive/demo_data/linear_orders_raw.json')

df_orders = df_orders['result']['records']
df_orders = pd.json_normalize(df_orders)

# Display the first few rows
df_orders.head()

,OrderEntryTime,1V,3P,3V,DeliveryEnd,OrderEntryUser,OrderPeriodID,BiddingLevelName,OrderID,2V,...,4P,DeliveryStart,SettlementCurrency,MarketName,EFA,TradeID,_full_text,Portfolio,_id,1P
0,2021-09-09T09:05:00,1100,17,0,2021-09-16T03:00:00,NGESO-A,100000000124255,DCL,100000000103201,1100,...,999.99,2021-09-15T23:00:00,GBP,DCL,1,100000000124255,"'-09':11,12,24,30 '-15':25 '-16':31 '0':36,41,...",NGESO-T001,1,0
1,2021-09-09T09:05:00,1100,17,0,2021-09-16T07:00:00,NGESO-A,100000000124257,DCL,100000000103201,1100,...,999.99,2021-09-16T03:00:00,GBP,DCL,2,100000000124257,"'-09':11,12,24,30 '-16':25,31 '0':36,41,43 '00...",NGESO-T001,2,0
2,2021-09-09T09:05:00,1100,17,0,2021-09-16T11:00:00,NGESO-A,100000000124259,DCL,100000000103201,1100,...,999.99,2021-09-16T07:00:00,GBP,DCL,3,100000000124259,"'-09':11,12,24,30 '-16':25,31 '0':36,41,43 '00...",NGESO-T001,3,0
3,2021-09-09T09:05:00,1100,17,0,2021-09-16T15:00:00,NGESO-A,100000000124261,DCL,100000000103201,1100,...,999.99,2021-09-16T11:00:00,GBP,DCL,4,100000000124261,"'-09':11,12,24,30 '-16':25,31 '0':36,41,43 '00...",NGESO-T001,4,0
4,2021-09-09T09:05:00,1100,17,0,2021-09-16T19:00:00,NGESO-A,100000000124263,DCL,100000000103201,1100,...,999.99,2021-09-16T15:00:00,GBP,DCL,5,100000000124263,"'-09':11,12,24,30 '-16':25,31 '0':36,41,43 '00...",NGESO-T001,5,0


In [24]:
print(len(df_bmrs))
print(len(df_orders))

1412
750


In [25]:
numerical_columns = ['initialForecastSpnGeneration', 'latestForecastSpnGeneration', 'fuelTypeGeneration']

# Convert columns to numeric, errors='coerce' will turn invalid values into NaN
df_bmrs[numerical_columns] = df_bmrs[numerical_columns].apply(pd.to_numeric, errors='coerce')

# Fill NULL (NaN) values with median of each column
df_bmrs[numerical_columns] = df_bmrs[numerical_columns].apply(lambda x: x.fillna(x.median()))
df_bmrs.head()

,recordType,startTimeOfHalfHrPeriod,settlementPeriod,initialForecastPublishingPeriodCommencingTime,initialForecastSpnGeneration,latestForecastPublishingPeriodCommencingTime,latestForecastSpnGeneration,outTurnPublishingPeriodCommencingTime,fuelTypeGeneration,activeFlag,EFA,combinedDatetime
0,WIND,2021-11-01,1,30/10/2021 03:30,9643.0,31/10/2021 22:30,9944.0,01/11/2021 00:30,10972.0,Y,1.0,2021-10-31T23:00:00
1,WIND,2021-11-01,2,NaN,7856.0,NaN,8610.0,01/11/2021 01:00,11053.0,Y,1.0,2021-10-31T23:30:00
2,WIND,2021-11-01,3,30/10/2021 03:30,9773.0,31/10/2021 22:30,9920.0,01/11/2021 01:30,11030.0,Y,1.0,2021-11-01T00:00:00
3,WIND,2021-11-01,4,NaN,7856.0,NaN,8610.0,01/11/2021 02:00,10787.0,Y,1.0,2021-11-01T00:30:00
4,WIND,2021-11-01,5,30/10/2021 03:30,9919.0,31/10/2021 22:30,10029.0,01/11/2021 02:30,10806.0,Y,1.0,2021-11-01T01:00:00


In [26]:
# Remove duplicates from both datasets
df_bmrs = df_bmrs.drop_duplicates()
df_orders = df_orders.drop_duplicates()

# Drop rows with any null values in both datasets
df_bmrs = df_bmrs.dropna()
df_orders = df_orders.dropna()

print(len(df_bmrs))
print(len(df_orders))

612
750


In [28]:
#join
df_merged = pd.merge(df_bmrs, df_orders,
                     left_on='combinedDatetime',
                     right_on='DeliveryStart',
                     how='inner')

print(len(df_merged))
df_merged.head()

256


,recordType,startTimeOfHalfHrPeriod,settlementPeriod,initialForecastPublishingPeriodCommencingTime,initialForecastSpnGeneration,latestForecastPublishingPeriodCommencingTime,latestForecastSpnGeneration,outTurnPublishingPeriodCommencingTime,fuelTypeGeneration,activeFlag,...,4P,DeliveryStart,SettlementCurrency,MarketName,EFA_y,TradeID,_full_text,Portfolio,_id,1P
0,WIND,2021-11-01,1,30/10/2021 03:30,9643.0,31/10/2021 22:30,9944.0,01/11/2021 00:30,10972.0,Y,...,NA,2021-10-31T23:00:00,GBP,DCH,1,100000000159427,"'-01':31 '-10':11,24 '-11':30 '-29':12 '-31':2...",NGESO-T001,427,0
1,WIND,2021-11-01,1,30/10/2021 03:30,9643.0,31/10/2021 22:30,9944.0,01/11/2021 00:30,10972.0,Y,...,999.99,2021-10-31T23:00:00,GBP,DCL,1,100000000159415,"'-01':31 '-10':11,24 '-11':30 '-29':12 '-31':2...",NGESO-T001,433,0
2,WIND,2021-11-01,9,30/10/2021 03:30,10441.0,31/10/2021 22:30,10740.0,01/11/2021 04:30,11269.0,Y,...,NA,2021-11-01T03:00:00,GBP,DCH,2,100000000159429,"'-01':25,31 '-10':11 '-11':24,30 '-29':12 '0':...",NGESO-T001,428,0
3,WIND,2021-11-01,9,30/10/2021 03:30,10441.0,31/10/2021 22:30,10740.0,01/11/2021 04:30,11269.0,Y,...,999.99,2021-11-01T03:00:00,GBP,DCL,2,100000000159417,"'-01':25,31 '-10':11 '-11':24,30 '-29':12 '0':...",NGESO-T001,434,0
4,WIND,2021-11-01,17,30/10/2021 03:30,11838.0,31/10/2021 22:30,12655.0,01/11/2021 08:30,12492.0,Y,...,NA,2021-11-01T07:00:00,GBP,DCH,3,100000000159431,"'-01':25,31 '-10':11 '-11':24,30 '-29':12 '0':...",NGESO-T001,429,0


In [29]:
# Save the DataFrame to a CSV file
df_merged.to_csv('df_merged.csv', index=False)

print("File has been saved as df_merged.csv")

File has been saved as df_merged.csv


In [ ]:
######################################################

In [ ]:
# Convert 'startTimeOfHalfHrPeriod' in CSV dataset to datetime
df_bmrs['startTimeOfHalfHrPeriod'] = pd.to_datetime(df_bmrs['startTimeOfHalfHrPeriod'], errors='coerce')
df_bmrs['startTimeOfHalfHrPeriodDate'] = df_bmrs['startTimeOfHalfHrPeriod'].dt.date

# Convert 'DeliveryStart' in JSON dataset to datetime
df_orders['DeliveryStart'] = pd.to_datetime(df_orders['DeliveryStart'], errors='coerce')
df_orders['DeliveryStartDate'] = df_orders['DeliveryStart'].dt.date

print(len(df_bmrs))
print(len(df_orders))

1415
750


In [ ]:
# Remove duplicates from both datasets
df_bmrs = df_bmrs.drop_duplicates()
df_orders = df_orders.drop_duplicates()

# Drop rows with any null values in both datasets
df_bmrs = df_bmrs.dropna()
df_orders = df_orders.dropna()

print(len(df_bmrs))
print(len(df_orders))

234
750


In [ ]:
#join
df_merged = pd.merge(df_bmrs, df_orders,
                     left_on='startTimeOfHalfHrPeriodDate',
                     right_on='DeliveryStartDate',
                     how='inner')

print(len(df_merged))
df_merged.head()


1152


,recordType,startTimeOfHalfHrPeriod,settlementPeriod,initialForecastPublishingPeriodCommencingTime,initialForecastSpnGeneration,latestForecastPublishingPeriodCommencingTime,latestForecastSpnGeneration,outTurnPublishingPeriodCommencingTime,fuelTypeGeneration,activeFlag,...,DeliveryStart,SettlementCurrency,MarketName,EFA_y,TradeID,_full_text,Portfolio,_id,1P,DeliveryStartDate
0,WIND,2021-11-01,1,01/11/2021 02:30,1935.0,02/11/2021 22:30,1846.0,03/11/2021 00:30,1695.0,Y,...,2021-11-01 03:00:00,GBP,DCH,2,100000000159429,"'-01':25,31 '-10':11 '-11':24,30 '-29':12 '0':...",NGESO-T001,428,0,2021-11-01
1,WIND,2021-11-01,1,01/11/2021 02:30,1935.0,02/11/2021 22:30,1846.0,03/11/2021 00:30,1695.0,Y,...,2021-11-01 07:00:00,GBP,DCH,3,100000000159431,"'-01':25,31 '-10':11 '-11':24,30 '-29':12 '0':...",NGESO-T001,429,0,2021-11-01
2,WIND,2021-11-01,1,01/11/2021 02:30,1935.0,02/11/2021 22:30,1846.0,03/11/2021 00:30,1695.0,Y,...,2021-11-01 11:00:00,GBP,DCH,4,100000000159433,"'-01':25,31 '-10':11 '-11':24,30 '-29':12 '0':...",NGESO-T001,430,0,2021-11-01
3,WIND,2021-11-01,1,01/11/2021 02:30,1935.0,02/11/2021 22:30,1846.0,03/11/2021 00:30,1695.0,Y,...,2021-11-01 15:00:00,GBP,DCH,5,100000000159435,"'-01':25,31 '-10':11 '-11':24,30 '-29':12 '0':...",NGESO-T001,431,0,2021-11-01
4,WIND,2021-11-01,1,01/11/2021 02:30,1935.0,02/11/2021 22:30,1846.0,03/11/2021 00:30,1695.0,Y,...,2021-11-01 19:00:00,GBP,DCH,6,100000000159437,"'-01':25,31 '-10':11 '-11':24,30 '-29':12 '0':...",NGESO-T001,432,0,2021-11-01


In [ ]:
##Build the feature engineering transformation

In [ ]:
df = df_merged

df['startTimeOfHalfHrPeriod'] = pd.to_datetime(df['startTimeOfHalfHrPeriod'])
df['OrderEntryTime'] = pd.to_datetime(df['OrderEntryTime'])
df['DeliveryStart'] = pd.to_datetime(df['DeliveryStart'])
df['DeliveryEnd'] = pd.to_datetime(df['DeliveryEnd'])
df['startTimeOfHalfHrPeriodDate'] = pd.to_datetime(df['startTimeOfHalfHrPeriodDate'])
df['initialForecastPublishingPeriodCommencingTime'] = pd.to_datetime(df['initialForecastPublishingPeriodCommencingTime'])
df['latestForecastPublishingPeriodCommencingTime'] = pd.to_datetime(df['latestForecastPublishingPeriodCommencingTime'])
df['outTurnPublishingPeriodCommencingTime'] = pd.to_datetime(df['outTurnPublishingPeriodCommencingTime'])
df['date'] = pd.to_datetime(df['date'])


In [ ]:
# Set the time-based index
df = df.sort_values(by='startTimeOfHalfHrPeriod')
df.set_index('startTimeOfHalfHrPeriod', inplace=True)

# Calculate the rolling median over a 6-hour window for 'initialForecastSpnGeneration' and 'ExecutedVolume'
df['rolling_median_initialForecastSpnGeneration'] = df['initialForecastSpnGeneration'].rolling('6H').median()
df['rolling_median_ExecutedVolume'] = df['ExecutedVolume'].rolling('6H').median()


<ipython-input-50-f792f5bd8001>:6: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df['rolling_median_initialForecastSpnGeneration'] = df['initialForecastSpnGeneration'].rolling('6H').median()
<ipython-input-50-f792f5bd8001>:7: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df['rolling_median_ExecutedVolume'] = df['ExecutedVolume'].rolling('6H').median()


In [ ]:
#Not sure how to proceed further.. need some more clarity on the dataset side and the relation